## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [1]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

#sys.path

In [2]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

c:\Users\Surface pro\anaconda3\lib\site-packages\mass_ts\_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Reading Datasets

In [3]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [4]:
df_other_methods['method'].unique()

array(['BOSS', 'Catch22', 'cBOSS', 'HIVE-COTEv1', 'InceptionTime',
       'ProximityForest', 'ResNet', 'RISE', 'ROCKET', 'S-BOSS', 'SAST',
       'STC', 'TS-CHIEF', 'TSF', 'WEASEL'], dtype=object)

In [5]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,1.000000,5.898681,0.081569,4.333925,1.479862,0.002375,Coffee,ACF&PACF: n_random_points=1 nb_inst_per_class=...,1,(max instances per class)//2,Rsast,ACF&PACF
1,1,0.964286,27.529371,0.090405,0.858840,26.572848,0.005367,Coffee,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,(lenthg ts)//2,1,Rsast,ACF&PACF
2,2,1.000000,203.339974,0.066815,4.774397,198.422380,0.068660,Coffee,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,(lenthg ts)//2,(max instances per class)//2,Rsast,ACF&PACF


In [6]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")

df_result['dataset_name'].unique()


array(['Coffee', 'DistalPhalanxOutlineAgeGroup',
       'DistalPhalanxOutlineCorrect', 'GunPoint', 'ItalyPowerDemand',
       'ShapeletSim', 'SmoothSubspace', 'SonyAIBORobotSurface1',
       'SonyAIBORobotSurface2', 'SyntheticControl'], dtype=object)

In [7]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [8]:

filter_param=df_result[df_result.rpoint=='100']
filter_param.rpoint.unique()

array(['100'], dtype=object)

In [9]:

filter_param=filter_param[filter_param.nb_per_class=='100']
filter_param.nb_per_class.unique()

array(['100'], dtype=object)

In [10]:
# create a pivot table with the mean of score by dataset

len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('dataset',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,score,method
0,DistalPhalanxOutlineAgeGroup,0.750600,0.721823,0.755396,0.714628,0.705036,0.690647,0.748201,ROCKET
1,DistalPhalanxOutlineCorrect,0.657005,0.723430,0.665459,0.699275,0.741546,0.713768,0.771739,ROCKET
2,GunPoint,0.815556,0.980000,0.857778,0.973333,0.973333,0.977778,1.000000,ROCKET
3,ItalyPowerDemand,0.954001,0.948170,0.955620,0.945254,0.952057,0.949466,0.968902,ROCKET
4,ShapeletSim,0.985185,0.883333,0.985185,0.820370,0.551852,0.883333,1.000000,ROCKET
5,SmoothSubspace,0.937778,0.922222,0.922222,0.926667,0.917778,0.942222,0.980000,ROCKET
6,SonyAIBORobotSurface2,0.855894,0.915005,0.856593,0.879328,0.859741,0.913956,0.916055,ROCKET
7,SyntheticControl,0.937778,0.970000,0.932222,0.967778,0.988889,0.972222,1.000000,ROCKET


In [11]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.838822,0.753602,0.811327,0.835575,0.832873,0.832464
ACF&PACF,0.907424,0.854165,0.874603,0.879545,0.876631,0.877997
Max ACF,0.846077,0.746052,0.813672,0.839253,0.832605,0.841706
Max PACF,0.880508,0.808671,0.855327,0.855068,0.858329,0.854180
None,0.876019,0.800264,0.828122,0.845196,0.830029,0.846214
PACF,0.907669,0.857110,0.872538,0.878449,0.881679,0.878214


In [12]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.838270,0.718345,0.816564,0.846858,0.842405,0.842320
ACF&PACF,0.909279,0.835083,0.870573,0.884467,0.885253,0.885382
Max ACF,0.839476,0.722752,0.818173,0.849218,0.846285,0.844627
Max PACF,0.881973,0.790017,0.855263,0.856414,0.867500,0.860658
None,0.881692,0.769501,0.833685,0.843992,0.851271,0.844703
PACF,0.907663,0.847752,0.872536,0.882011,0.882163,0.883535


#### Time: Subsequence Lenght Method 

In [13]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,Coffee,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SonyAIBORobotSurface1,SonyAIBORobotSurface2,SyntheticControl
len_method,,,,,,,,,,
ACF,63.990256,111.545551,144.720377,37.041264,0.353901,7.361113,0.561044,0.190215,0.700003,90.876566
ACF&PACF,194.147516,294.637278,356.864070,71.945396,0.729001,146.904885,0.930945,1.326816,2.032898,211.081948
Max ACF,4.597660,40.968521,47.509385,4.165960,0.361998,2.827787,0.565765,0.176733,0.548111,32.034245
Max PACF,7.392416,19.410193,21.999060,4.032023,0.387409,66.417229,0.639174,0.245754,0.413045,16.692008
None,2.293117,18.884042,21.533008,2.376824,0.331860,1.788605,0.523768,0.147382,0.305284,15.885983
PACF,159.124152,202.722461,237.791324,47.107583,0.711325,140.841731,0.843548,1.022764,1.655537,160.264361


In [14]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,8.570878,2.320997,16.416915,98.575851,44.170943,65.159331
ACF&PACF,21.082490,6.167764,56.786291,259.115506,123.305627,175.116897
Max ACF,0.993336,1.241650,5.295414,31.890666,13.504895,20.460754
Max PACF,1.341083,1.923753,12.328640,23.905234,15.905865,18.751630
None,0.651917,0.962070,2.783531,14.987753,6.500898,9.457833
PACF,15.313166,4.900241,44.500816,185.776818,90.781362,127.293928


In [15]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,14.521095,2.123891,13.545480,98.013000,39.765676,66.195735
ACF&PACF,36.053439,12.676521,46.670054,236.019062,121.369543,186.144024
Max ACF,1.514642,1.074464,4.930948,30.763075,13.231155,21.780436
Max PACF,1.859312,8.647510,10.762880,18.753758,15.756020,18.285275
None,0.938350,0.876539,2.951851,12.885844,7.194875,10.445994
PACF,26.753471,11.312090,35.460091,163.499973,92.025098,137.496730


#### Variance: Subsequence Lenght Method 

In [16]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.008940,0.018107,0.011184,0.011923,0.010876,0.011231
ACF&PACF,0.004939,0.009880,0.008590,0.009177,0.008890,0.009186
Max ACF,0.010700,0.018072,0.011761,0.011914,0.010009,0.012031
Max PACF,0.008189,0.020595,0.008890,0.011424,0.009513,0.010072
None,0.007904,0.019333,0.016376,0.017344,0.017560,0.018199
PACF,0.004753,0.009722,0.008477,0.010013,0.009227,0.009672


In [17]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.008631,0.018197,0.013317,0.012823,0.013022,0.013373
ACF&PACF,0.004831,0.011209,0.008732,0.009038,0.009381,0.009098
Max ACF,0.008970,0.021507,0.013509,0.012664,0.011263,0.011666
Max PACF,0.008216,0.015904,0.011796,0.011976,0.010636,0.012513
None,0.008641,0.020307,0.018638,0.016678,0.019067,0.017091
PACF,0.004781,0.010922,0.009224,0.009304,0.008716,0.009445


#### Another Results

In [18]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,5040,5040,5040
unique,10,6,216
top,GunPoint,ACF&PACF,Max ACF: n_random_points= (lenthg ts)//2 nb_in...
freq,720,840,80


In [19]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
Coffee,1.798105
DistalPhalanxOutlineAgeGroup,14.336834
DistalPhalanxOutlineCorrect,17.300359
GunPoint,5.555635
ItalyPowerDemand,0.095850
ShapeletSim,7.627945
SmoothSubspace,0.135475
SonyAIBORobotSurface1,0.038871
SonyAIBORobotSurface2,0.188496


In [20]:
df_result['time'].sum()/(60*60)

58.053300451835

#### Generate CD Diagram Tunning Hyperparameter

In [21]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()


In [22]:
param_per_ds=pd.pivot_table(df_perf, index=['dataset_name'],values=['classifier_name'] ,aggfunc='count')
param_per_ds=param_per_ds.reset_index()
complete_ds=param_per_ds[param_per_ds.classifier_name==max(param_per_ds.classifier_name)]["dataset_name"].unique()

df_perf=df_perf[df_perf.dataset_name.isin(complete_ds)]
df_result=df_result[df_result.dataset_name.isin(complete_ds)]
df_perf.dataset_name.unique()

array(['GunPoint', 'ItalyPowerDemand', 'SmoothSubspace',
       'SonyAIBORobotSurface2'], dtype=object)

In [23]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='images_cd_diagram/cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats

In [24]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,0
ACF: n_random_points=1 nb_inst_per_class=1,214.250
Max ACF: n_random_points=1 nb_inst_per_class=1,214.250
ACF: n_random_points=10 nb_inst_per_class=1,209.000
Max PACF: n_random_points=1 nb_inst_per_class=1,208.375
None: n_random_points=1 nb_inst_per_class=1,206.500
...,...
PACF: n_random_points=30 nb_inst_per_class=50,36.375
PACF: n_random_points=100 nb_inst_per_class=100,35.125
PACF: n_random_points=50 nb_inst_per_class=100,33.500
ACF&PACF: n_random_points=100 nb_inst_per_class=50,31.375


In [25]:
#generate example cd diagrams
#draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

#### Focus on most accurate lenght method

##### Overall Accuracy

In [26]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=50'], dtype='object')

In [27]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.933242,0.870266,0.915204,0.941886,0.943716,0.948270
1,0.927002,0.768364,0.898312,0.918624,0.926859,0.909377
10,0.927856,0.868657,0.919051,0.939790,0.939651,0.941335
100,0.938064,0.884688,0.924395,0.941349,0.941673,0.946673
30,0.938042,0.886199,0.920385,0.943795,0.937354,0.943178
50,0.930096,0.890218,0.926359,0.943884,0.941349,0.942935


In [28]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.001063,0.005687,0.002171,0.000964,0.000840,0.000776
1,0.001305,0.022783,0.004532,0.002178,0.001955,0.003764
10,0.001619,0.005706,0.002192,0.000832,0.000844,0.000589
100,0.000984,0.005550,0.001781,0.000785,0.000718,0.000603
30,0.001051,0.007255,0.002199,0.000836,0.001052,0.000718
50,0.001510,0.001855,0.001902,0.000617,0.000874,0.000667


##### Overall time complexity

In [29]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
GunPoint,0.000645,0.027494,0.000965,1.169863,1.199090
ItalyPowerDemand,0.000136,0.001336,0.000842,0.009811,0.012150
SmoothSubspace,0.000103,0.002251,0.002864,0.010263,0.015516
SonyAIBORobotSurface2,0.000321,0.002770,0.000283,0.030471,0.033882


In [30]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["(max instances per class)//2","1","10","30","50","100"], index=["(lenthg ts)//2","1","10","30","50","100"])
pivot

nb_per_class,(max instances per class)//2,1,10,30,50,100
rpoint,,,,,,
(lenthg ts)//2,0.366423,0.034077,0.289694,0.712369,0.716229,0.731966
1,0.044491,0.009157,0.016136,0.028492,0.025892,0.026516
10,0.058035,0.013833,0.055648,0.125056,0.128500,0.128659
30,0.157187,0.020914,0.142794,0.346109,0.349427,0.345032
50,0.248416,0.030580,0.229786,0.558682,0.565004,0.565693
100,0.452801,0.040076,0.430480,1.045892,1.052147,1.048486


##### Calculate weights time complexity

In [31]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.018074,0.019139,0.021926,0.015188,0.022034,0.018167
1,0.024291,0.017271,0.019348,0.019141,0.021121,0.022279
10,0.018494,0.020264,0.019207,0.013734,0.014345,0.017209
100,0.016501,0.017553,0.014360,0.017920,0.014780,0.018081
30,0.020793,0.018451,0.016424,0.017510,0.016753,0.017044
50,0.020330,0.015078,0.017862,0.018102,0.012056,0.019661


##### Finding subsequences time complexity

In [32]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.561489,0.058876,0.507546,1.062333,1.042694,1.002236
1,1.223498,0.095188,0.252778,0.618989,0.526851,0.577188
10,0.507399,0.029899,0.252129,0.469564,0.450308,0.544677
100,0.529924,0.031332,0.260453,0.742328,0.623679,0.737281
30,0.515034,0.030087,0.263972,0.655696,0.567569,0.628217
50,0.556127,0.024119,0.272634,0.636393,0.534400,0.671816


##### Transform Dataset time complexity

In [33]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,21.363305,1.713831,16.827922,42.789189,41.626904,41.896007
1,1.073418,0.195119,0.442162,0.938418,0.913739,0.937944
10,2.941512,0.494439,3.051443,7.219708,7.024177,7.133466
100,26.588871,2.186144,25.528242,62.103425,62.067875,62.326692
30,8.880329,1.016662,8.268483,20.009024,20.160730,20.298974
50,14.307272,1.596156,13.476441,33.260166,32.952595,33.180834


##### Classifier time complexity

In [34]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.034953,0.251880,0.020306,0.044565,0.045168,0.051725
1,0.347032,0.240628,0.250880,0.013227,0.245205,0.014150
10,0.013490,0.283628,0.014283,0.014773,0.012608,0.012744
100,0.029821,0.167848,0.023013,0.041625,0.043333,0.043195
30,0.012557,0.188222,0.014345,0.017251,0.019125,0.019168
50,0.019317,0.197961,0.016173,0.024397,0.018625,0.025202
